<a href="https://colab.research.google.com/github/Leoli04/llms-notebooks/blob/main/huggingface/hf_nlp_08_troubleshoot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## troubleshoot

 如何调试执行trainer.train() 时，遇到的错误

### 调试训练管道
当您在 trainer.train() 中遇到错误时，问题是它可能来自多个来源，因为 Trainer 通常会将很多东西放在一起。
- **数据集问题**：它将数据集转换为数据加载器，因此问题可能是数据集中出现问题；
- **模型代码问题**：或者尝试将数据集的元素批处理在一起时出现问题。然后它会获取一批数据并将其输入模型，因此问题可能出在模型代码中。
- **优化器问题**：它计算梯度并执行优化步骤，因此问题也可能出在您的优化器中。
- **评估指标问题**：即使训练一切顺利，如果您的指标存在问题，评估过程中仍然可能会出现问题。

我们将使用以下脚本（尝试）在 [MNLI 数据集](https://huggingface.co/datasets/glue)上微调 DistilBERT 模型：

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
!pip install transformers[torch] -U

In [1]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model,
    args,
    train_dataset=raw_datasets["train"],
    eval_dataset=raw_datasets["validation_matched"],
    compute_metrics=compute_metrics,
)
trainer.train()

Map:   0%|          | 0/9815 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ValueError: You have to specify either input_ids or inputs_embeds

执行上述代码，会得到如下错误：
```
---> 47 trainer.train()
……
ValueError: You have to specify either input_ids or inputs_embeds

```

#### 检查数据集

为了避免花费无数时间尝试修复不是错误根源的问题，我们**建议您使用 trainer.train_dataset 进行检查**，而不使用其他任何东西。那么让我们在这里这样做：

In [2]:
trainer.train_dataset[0]

{'premise': 'Conceptually cream skimming has two basic dimensions - product and geography.',
 'hypothesis': 'Product and geography are what make cream skimming work. ',
 'label': 1,
 'idx': 0}

你注意到有什么不对劲吗？结合有关 input_ids 缺失的错误消息，您应该意识到这些是文本，而不是模型可以理解的数字。在这里，原始错误非常具有误导性，因为 Trainer 自动删除与模型签名不匹配的列（即模型期望的参数）。这意味着在这里，除了标签之外的所有东西都被丢弃了。因此，创建批次然后将它们发送到模型没有问题，模型又抱怨它没有收到正确的输入。

为什么数据没有被处理？我们确实在数据集上使用了 Dataset.map() 方法来对每个样本应用分词器。但是如果你仔细观察代码，你会发现我们在将训练集和评估集传递给 Trainer 时犯了一个错误。我们在这里没有使用 tokenized_datasets ，而是使用 raw_datasets 🤦。所以让我们解决这个问题！

In [3]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model,
    args,
    # train_dataset=raw_datasets["train"],
    # eval_dataset=raw_datasets["validation_matched"],
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
)
trainer.train()

Map:   0%|          | 0/9832 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: expected sequence of length 58 at dim 1 (got 47)

重新执行上述代码，会得到如下错误：
```
/usr/local/lib/python3.10/dist-packages/transformers/data/data_collator.py in torch_default_data_collator(features)
    156                 batch[k] = torch.tensor(np.stack([f[k] for f in features]))
    157             else:
--> 158                 batch[k] = torch.tensor([f[k] for f in features])
    159
    160     return batch
ValueError: expected sequence of length 58 at dim 1 (got 47)
```

从上面错误可以看出，问题出现在数据整理data_collator。在查看数据整理之前，让我们检查一下我们的数据，以确保 100% 确定它是正确的。

In [4]:
# 查看模型的解码输入：使用分词器来解码输入
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'[CLS] conceptually cream skimming has two basic dimensions - product and geography. [SEP] product and geography are what make cream skimming work. [SEP]'

In [5]:
# 查看输入的建
trainer.train_dataset[0].keys()

dict_keys(['premise', 'hypothesis', 'label', 'idx', 'input_ids', 'attention_mask'])

> 请注意，与模型接受的输入不对应的键将被自动丢弃，因此这里我们仅保留 input_ids 、 attention_mask 和 label （将被重命名为 labels ）。

In [7]:
# 检查模型：到模型卡页面，查看模型接受的参数
type(trainer.model)

transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification

In [8]:
# 查看模型注意力
trainer.train_dataset[0]["attention_mask"]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [9]:
# 检查注意掩码 长度与 输入 ID 的长度是否相同
len(trainer.train_dataset[0]["attention_mask"]) == len(
    trainer.train_dataset[0]["input_ids"]
)

True

In [10]:
# 检查标签
trainer.train_dataset[0]["label"]

1

In [11]:
# 查看标签对应的name信息
trainer.train_dataset.features["label"].names

['entailment', 'neutral', 'contradiction']

所以 1 意味着 neutral 。

这里没有令牌类型 ID，因为 DistilBERT 不需要它们；如果您的模型中有一些，您还应该确保它们正确匹配输入中第一句和第二句的位置。

以相同的方式仔细检查验证集和测试集。

#### 从数据集到数据加载器

训练管道中下一个可能出错的事情是当 Trainer 尝试从训练或验证集形成批次时。一旦确定 Trainer 的数据集正确，您可以尝试通过执行以下命令来手动形成批次（将 train 替换为 eval 进行验证数据加载器）：

此代码创建训练数据加载器，然后迭代它，在第一次迭代时停止。如果代码执行时没有错误，您就有了可以检查的第一个训练批次，如果代码出错，您可以确定问题出在数据加载器中

In [12]:
for batch in trainer.get_train_dataloader():
    break

ValueError: expected sequence of length 58 at dim 1 (got 47)

根据报错堆栈信息`in torch_default_data_collator`可以大概知道问题。

批处理创建过程中的大多数问题都是由于将示例整理到单个批处理中而出现的，因此当有疑问时首先要检查的是 collate_fn 您的 DataLoader 正在使用什么：

In [13]:
data_collator = trainer.get_train_dataloader().collate_fn
data_collator

<function transformers.data.data_collator.default_data_collator(features: List[transformers.data.data_collator.InputDataClass], return_tensors='pt') -> Dict[str, Any]>

用的default_data_collator，但这不是我们在本例中想要的。我们希望将示例填充到批次中最长的句子，这是由 DataCollatorWithPadding 整理器完成的。这个数据整理器应该被 Trainer 默认使用的，那么为什么这里不使用它呢？


检查trainer构造处的代码可以发现，因为我们没有将 tokenizer 传递给 Trainer ，所以它无法创建我们想要的 DataCollatorWithPadding 。在实践中，您应该毫不犹豫地显式传递您想要使用的数据整理器，以确保避免此类错误。


在Trainer增加如下代码：
```
 data_collator=data_collator,
 tokenizer=tokenizer,
```

#### 浏览模型

在重新之前代码之前，我们先来确定下数据整理器是正确的。尝试将其应用于数据集的几个样本：

In [15]:
data_collator = trainer.get_train_dataloader().collate_fn
batch = data_collator([trainer.train_dataset[i] for i in range(4)])

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`premise` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

此代码将失败，因为 train_dataset 包含字符串列，而 Trainer 通常会删除这些字符串列。您可以手动删除它们，或者调用 Trainer._remove_unused_columns() 方法来执行此操作：

In [16]:
data_collator = trainer.get_train_dataloader().collate_fn
actual_train_set = trainer._remove_unused_columns(trainer.train_dataset)
batch = data_collator([actual_train_set[i] for i in range(4)])

In [17]:
# 重新验证数据加载器
for batch in trainer.get_train_dataloader():
    break

> 如果在笔记本中运行此代码，遇到 CUDA 错误，需要重新启动笔记本并重新执行不带 trainer.train() 行的最后一个代码段。这是因为它们会不可挽回地破坏你的内核。

这与 GPU 的工作方式有关。它们在并行执行大量操作方面非常高效，但缺点是当其中一条指令导致错误时，您无法立即知道。只有当程序调用 GPU 上的多个进程的同步时，它才会意识到出了问题，因此错误实际上是在与创建它的原因无关的地方引发的。例如，如果我们查看之前的错误栈，就会发现错误是在向后传递过程中引发的，但我们很快就会看到它实际上源于向前传递中的某些内容。

如何避开这类问题呢？我们只需将模型放回 CPU 并在批处理中调用它 - DataLoader 返回的批处理尚未移至 GPU：

In [ ]:
outputs = trainer.model.cpu()(**batch)

In [14]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

Map:   0%|          | 0/9796 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


IndexError: Target 2 is out of bounds.

重新运行上述代码：
```
IndexError: Target 2 is out of bounds.
```
在损失计算中出现了 IndexError ，更准确地说，我们可以看到是目标 2 产生了错误，因此这是检查模型标签数量的好时机：

In [18]:
trainer.model.config.num_labels

2

对于两个标签，只允许 0 和 1 作为目标，但是根据错误消息，我们得到了 2。得到 2 实际上是正常的：如果我们还记得之前提取的标签名称，则有 3 个，因此索引为 0 、 1 和 2 在我们的数据集中。问题是我们没有告诉我们的模型，它应该是用三个标签创建的。所以我们需要在模型实例化的时候，传递这个参数。


In [19]:
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
# model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
# trainer.train()

Map:   0%|          | 0/9847 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


上面的步骤没有包含 trainer.train() 行，再执行训练之前先检查数据加载、并将数据传递给模型。

In [20]:
# 在cpu上检查：加载数据，并将数据传递给模型
for batch in trainer.get_train_dataloader():
    break

outputs = trainer.model.cpu()(**batch)

In [21]:
import torch
# 返回 GPU 检查：加载数据，并将数据传递给模型

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: v.to(device) for k, v in batch.items()}

outputs = trainer.model.to(device)(**batch)

#### 执行一项优化步骤

In [22]:
loss = outputs.loss
loss.backward()

要执行优化步骤，我们只需要创建 optimizer 并调用其 step() 方法。

如果您有自定义优化器，则此处可能会出现一些需要调试的问题。

In [23]:
trainer.create_optimizer()
trainer.optimizer.step()

#### 处理 CUDA 内存不足错误

每当您收到以 RuntimeError: CUDA out of memory 开头的错误消息时，这表明您的 GPU 内存不足。此错误意味着您尝试在 GPU 内存中放入太多内容，从而导致错误。与其他 CUDA 错误一样，您需要重新启动内核才能再次运行训练。

要解决这个问题，您只需要使用更少的 GPU 空间：
- 首先，确保 GPU 上没有同时有两个模型（当然，除非您的问题需要这样做）。
- 然后，您可能应该减少批量大小，因为它直接影响模型的所有中间输出及其梯度的大小。
- 如果问题仍然存在，请考虑使用较小版本的模型。

#### 评估模型

现在我们已经解决了代码的所有问题，执行trainer.train() 命令

In [ ]:
trainer.train()

运行trainer.train() 命令，得到如下错误：
```
TypeError: only size-1 arrays can be converted to Python scalars
```

这个错误出现在评估阶段，您可以独立于训练运行 Trainer 的评估循环，如下所示：

In [ ]:
trainer.evaluate()

Epoch,Training Loss,Validation Loss


> 您应该始终确保在启动 trainer.train() 之前可以运行 trainer.evaluate() ，以避免在发生错误之前浪费大量计算资源。

在尝试调试评估循环中的问题之前，先检查数据，能够正确形成批次，并且可以在其上运行模型。

In [26]:
for batch in trainer.get_eval_dataloader():
    break

batch = {k: v.to(device) for k, v in batch.items()}

with torch.no_grad():
    outputs = trainer.model(**batch)

错误稍后出现，在评估阶段结束时，如果我们查看回溯，我们会看到以下内容：
```
~/git/datasets/src/datasets/metric.py in add_batch(self, predictions, references)
    431         """
    432         batch = {"predictions": predictions, "references": references}
--> 433         batch = self.info.features.encode_batch(batch)
    434         if self.writer is None:
    435             self._init_writer()
```

这告诉我们错误源自 datasets/metric.py 模块 - 所以这是我们的 compute_metrics() 函数的问题。它需要一个包含 logits 和标签的元组作为 NumPy 数组，所以让我们尝试向它提供：

In [ ]:
predictions = outputs.logits.cpu().numpy()
labels = batch["labels"].cpu().numpy()

compute_metrics((predictions, labels))

我们得到了同样的错误，所以问题肯定出在该函数上。

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


compute_metrics((predictions, labels))

#### 完整代码

In [ ]:
import numpy as np
from datasets import load_dataset
import evaluate
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)

raw_datasets = load_dataset("glue", "mnli")

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def preprocess_function(examples):
    return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)


tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

args = TrainingArguments(
    f"distilbert-finetuned-mnli",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
)

metric = evaluate.load("glue", "mnli")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation_matched"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.train()

### 在训练期间调试静默错误

我们可以做些什么来调试一个没有错误但没有得到好的结果的训练？
我们将在这里为您提供一些指导，但请注意，这种调试是机器学习中最难的部分，并且没有神奇的答案。

#### 检查数据

只有当你的模型实际上可以从你的数据中学习任何东西时，你的模型才会学到一些东西。如果存在损坏数据的错误或随机分配标签，则您很可能无法在数据集上进行任何模型训练。因此，请务必首先仔细检查解码的输入和标签，然后问自己以下问题：

- 解码后的数据可以理解吗？
- 你同意这些标签吗？
- 有没有一个标签比其他标签更常见？
- 如果模型预测随机答案/始终相同的答案，损失/指标应该是多少？

#### 在一个批次上验证模型

尝试一遍又一遍地在一批上训练你的模型是一个很好的测试，可以检查你所构建的问题是否可以通过你尝试训练的模型来解决。它还将帮助您查看您的初始学习率是否太高。

一旦定义了 Trainer ，执行此操作非常简单；只需获取一批训练数据，然后仅使用该批次运行一个小型手动训练循环，执行大约 20 个步骤：

In [ ]:
for batch in trainer.get_train_dataloader():
    break

batch = {k: v.to(device) for k, v in batch.items()}
trainer.create_optimizer()

for _ in range(20):
    outputs = trainer.model(**batch)
    loss = outputs.loss
    loss.backward()
    trainer.optimizer.step()
    trainer.optimizer.zero_grad()

生成的模型应该在相同的 batch 上具有接近完美的结果。让我们计算结果预测的指标：

In [ ]:
with torch.no_grad():
    outputs = trainer.model(**batch)
preds = outputs.logits
labels = batch["labels"]

compute_metrics((preds.cpu().numpy(), labels.cpu().numpy()))

100% 准确率，现在这是完美匹配的一个很好的例子。如果你不能让你的模型获得这样的完美结果，这意味着你构建问题或数据的方式有问题，所以你应该修复它。

#### 在有了第一个基线之前不要调整任何东西

超参数调整总是被强调为机器学习中最难的部分，但这只是帮助您获得一点指标的最后一步。大多数时候， Trainer 的默认超参数可以很好地为您提供良好的结果，因此，在您获得超越基线的参数之前，不要启动耗时且成本高昂的超参数搜索。你的数据集上有。

一旦你有了足够好的模型，你就可以开始进行一些调整。不要尝试使用不同的超参数启动一千次运行，而是比较一个超参数具有不同值的几次运行，以了解哪一个影响最大。

如果您正在调整模型本身，请保持简单，并且不要尝试任何无法合理证明的事情。